In [ ]:
pip install selenium

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request
import re
import pandas as pd
import datetime
import os
import getpass
from selenium import webdriver

In [ ]:
import csv
import urllib 

In [ ]:
def get_URL(page, word):
    word = urllib.parse.quote(word)
    url_before_page="http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery="+word+"&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    url_after_page="&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query="+word
    URL=url_before_page+page+url_after_page
    return URL

In [ ]:
def get_link(csv_name, page_num):
   
    for i in range(page_num, page_num+2):
        current_page=i*10
        URL=get_URL(str(current_page), csv_name)
        source_code_from_URL = urllib.request.urlopen(URL)
        soup=bs(source_code_from_URL, 'lxml', from_encoding='utf-8')
        try: 
            page_link = soup.select("li > div.cont > p.title > a") # a tag 가져오기
            # print(page_link[9])
        except:
            page_link = ""
        for j in range(10):
          
          # print(page_link.attrs)
          # print(j," ", page_link[j].attrs['href'])
          page_url = "http://riss.or.kr"+page_link[j].attrs['href'] # a tag 속성값 href 가져오기
          # print(page_url)
          reference_data = get_reference(page_url)
          # reference_data = pd.DataFrame(reference_data, index=['저자','국문제목','영문제목','수록지','핵심어','국문 요약','영문 요약','링크'])
          save_csv(csv_name, reference_data)
         

In [ ]:
def get_reference(URL):
    # reference_data = []
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
    driver.get(URL)
    
    html=driver.page_source
    soup=bs(html, "html.parser")
    
    title = soup.find("h3","title")
    # print(title)
    title_txt = title.get_text("", strip=True).split('=')
    # title_txt = title.get_text("", strip=True)
    title_kor = re.sub("\n\b", "", str(title_txt[0]).strip())
    try:
      title_eng = str(title_txt[1]).strip()
    except:
      title_eng = "None"
    
    txt_box = []
    
    for text in soup.find_all("div", "text"):
        txt = text.get_text("", strip=True)
        txt_box.append(txt)
    # print(txt_box)
    try:
      txt_kor=txt_box[1]
    except:
      txt_kor = "None"
    try:
      txt_eng=txt_box[3]
    except:
      txt_eng = "None"
    
    author = "None"
    book = "None"
    keyword = "None"
    detail_box = []
    detail_info = soup.select("#soptionview > div > div > div.infoDetail.on > div.infoDetailL > ul > li > div > p") # 중간에 div 하나 더 추가
    # print(detail_info)
    for detail in detail_info:
        detail_concept=detail.get_text("", strip=True)
        
        detail_wrap=[]
        detail_wrap.append(detail_concept)
        
        detail_box.append(detail_wrap)
        
    author = ",".join(detail_box[0])
    
    book=(
        "".join(detail_box[2]+detail_box[3]).replace("\n", "").replace("\t","").replace(" ","")
        + "p."
        +"".join(detail_box[-2])
    )
    
    keyword=",".join(detail_box[6])
        
    reference_data = [
            author,
            title_kor,
            title_eng,
            book,
            keyword,
            txt_kor,
            txt_eng,
            URL,
    ]
    
    # print(reference_data)
    # reference_data = pd.DataFrame(reference_data, index=['저자','국문제목','영문제목','수록지','핵심어','국문 요약','영문 요약','링크'])   
    driver.close()
    
    return reference_data
    
    
    

In [ ]:
def save_csv(csv_name, data):
    # csv_path=csv_name+".csv"
    # csv = csv_path.replace("/","\\")
    
    # if os.path.isfile(csv_path):
    #     data.to_csv(csv, mode='a', header=False, index=False)
        
    # else:
    #     data.to_csv(csv, mode='w', header=True, index=False)
    f = open(csv_name+'.csv','a', newline='')
    wr = csv.writer(f)
    wr.writerow(data)
 
    f.close()

In [ ]:
def make_folder(folder_name):
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

In [ ]:
if __name__ == "__main__":
    # now = datetime.datetime.now().strftime("%Y-%m-%d")
    # user_name=getpass.getuser()
    # folder_root="your path"
    # path=folder_root+now
    # make_folder(path)
    
    #cvs_name=input("저장할 csv이름을 입력하세요: ")
    csv_name='블록체인'
    col_name = ['저자','국문제목','영문제목','수록지','핵심어','국문 요약','영문 요약','링크']
    save_csv(csv_name, col_name)
    page_num=9
    #input("크롤링할 페이지 수를 입력하세요: ")
    get_link(csv_name, page_num)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launch